In [4]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D

ValueError: Name tf.RaggedTensorSpec has already been registered for class tensorflow.python.ops.ragged.ragged_tensor.RaggedTensorSpec.

In [ ]:
# En esta sección ser cargan las imágenes a color de emojis. Las imágenes
# se orginizan en 5 directorios, los cuales son: Angry, Happy, Poo, Sad y Surprise.
# Los nombres de las imágenes no están estandarizadas, por lo que es necesario primero ver qué hay 
# en cada directorio y después cargarlas.

path = '/Users/josep/Downloads/Emojis'

emojis = []
labels = ['Angry', 'Happy', 'Poo', 'Sad', 'Surprised']
for label in labels:
    folder = os.path.join(path, label)
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                emojis.append((img, label))

print("Número total de imágenes cargadas:", len(emojis))

In [ ]:
# Muestra una imagen de cada categoría
plt.figure(figsize=(12, 8))
for i, label in enumerate(labels):
    for img, img_label in emojis:
        if img_label == label:
            plt.subplot(1, 5, i + 1)
            plt.imshow(img)
            plt.title(label)
            plt.axis('off')
            break

In [ ]:
# Preprocesamiento de las imágenes. Las imágenes las vamos a pasar a escala de grises, a un 
# tamaño de 32x32 píxeles. Desafortunadamente, los trazos no están centrados en la imagen, por lo que
# también es necesario encontrar la zona del trazo, y recortarlo para después centrarlo en la imagen.

def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Imagen binaria (fondo blanco, trazos negros)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        min_area = 30
        all_points = np.vstack([cnt for cnt in contours if cv2.contourArea(cnt) > min_area])
        x, y, w, h = cv2.boundingRect(all_points)
        roi = binary[y:y+h, x:x+w]  # Usar la imagen binaria
        roi_resized = cv2.resize(roi, (32, 32), interpolation=cv2.INTER_AREA)
        padded = np.pad(roi_resized, ((2, 2), (2, 2)), mode='constant', constant_values=0)
        return padded
    else:
        return cv2.resize(binary, (32, 32), interpolation=cv2.INTER_AREA)

preprocessed_emojis = [(preprocess_image(img), label) for img, label in emojis]
print("Número total de imágenes preprocesadas:", len(preprocessed_emojis))

In [ ]:
# Muestra una imagen de cada categoría preprocesada
plt.figure(figsize=(12, 8))
for i, label in enumerate(labels):
    for img, img_label in preprocessed_emojis:
        if img_label == label:
            plt.subplot(1, 5, i + 1)
            plt.imshow(img, cmap='gray')
            plt.title(label)
            plt.axis('off')
            break

In [ ]:
# Ajuste y evaluación de un modelo MLP con las imágenes binarias preprocesadas
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Preparar los datos
X_bin = np.array([img.flatten() for img, label in preprocessed_emojis])
y_bin = np.array([labels.index(label) for img, label in preprocessed_emojis])

# Normalizar
X_bin = X_bin.astype('float32') / 255.0

# Codificar etiquetas
num_classes = len(labels)
y_bin_cat = to_categorical(y_bin, num_classes=num_classes)

# Dividir en entrenamiento y prueba
Xb_train, Xb_test, yb_train, yb_test = train_test_split(X_bin, y_bin_cat, test_size=0.2, random_state=42, stratify=y_bin)

# Definir el modelo MLP
mlp_bin = Sequential()
mlp_bin.add(Dense(128, activation='relu', input_dim=X_bin.shape[1]))
mlp_bin.add(Dense(64, activation='relu'))
mlp_bin.add(Dense(num_classes, activation='softmax'))
mlp_bin.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
mlp_bin.fit(Xb_train, yb_train, epochs=15, batch_size=32, validation_split=0.2)

# Evaluar el modelo
yb_pred = mlp_bin.predict(Xb_test)
yb_pred_classes = np.argmax(yb_pred, axis=1)
yb_true_classes = np.argmax(yb_test, axis=1)
print(classification_report(yb_true_classes, yb_pred_classes, target_names=labels))

In [ ]:
# Ajuste y evaluación de una CNN para las imágenes binarias preprocesadas
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

# Preparar los datos para la CNN
X_cnn = np.array([img for img, label in preprocessed_emojis])
X_cnn = X_cnn.astype('float32') / 255.0
X_cnn = X_cnn[..., np.newaxis]  # Añadir canal para imágenes en escala de grises

y_cnn = np.array([labels.index(label) for img, label in preprocessed_emojis])
num_classes = len(labels)
y_cnn_cat = to_categorical(y_cnn, num_classes=num_classes)

# Dividir en entrenamiento y prueba
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_cnn, y_cnn_cat, test_size=0.2, random_state=42, stratify=y_cnn)

# Definir el modelo CNN
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(X_cnn.shape[1], X_cnn.shape[2], 1)))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(num_classes, activation='softmax'))
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
cnn.fit(Xc_train, yc_train, epochs=15, batch_size=32, validation_split=0.2)

# Evaluar el modelo
yc_pred = cnn.predict(Xc_test)
yc_pred_classes = np.argmax(yc_pred, axis=1)
yc_true_classes = np.argmax(yc_test, axis=1)
print(classification_report(yc_true_classes, yc_pred_classes, target_names=labels))